In [ ]:
#Load modules
import sys
import os
import json
import csv
from datetime import date

## Metrics

- how many measurement types are used (count by type)?
- how many device types have the same measurement type defined more than once?
- how many different topic / data key combinations exist for the same measurement type?

In [ ]:
#Get all measurement categories
libraryPath = '../types'
types = []
for manufacturer in os.listdir(libraryPath):
    manufacturer_path = libraryPath + "/" + manufacturer
    if os.path.isdir(manufacturer_path) and not manufacturer.startswith('.'):
        for dt in os.listdir(manufacturer_path):
            dt_path = manufacturer_path + "/" + dt
            if os.path.isdir(dt_path) and not dt.startswith('.'):
                types.append(dt_path)
                
print("number of types: "+ str(len(types)))

In [ ]:
from glob import glob
import pandas as pd

data_key_stats = []
measurement_type_stats = {}

for dt in types:
    for schema in glob(dt + "/*.schema.json", recursive = True):
        manufacturer = schema.split(os.sep)[2]
        dt = schema.split(os.sep)[3]
        topic = schema.split(os.sep)[4].split(".schema.json")[0]
        with open(schema) as f:
            dt_schema = json.load(f)
            n_data_keys = len(dt_schema["properties"].keys())
            data_key_stats.append({"n_data_keys": n_data_keys, "manufacturer": manufacturer, "device_type": dt, "topic": topic, "device_type_id": manufacturer + "." + dt})

            for data_key, value in dt_schema["properties"].items():
                measurement_type = value.get("$ref")
                if measurement_type is not None:
                    if measurement_type in measurement_type_stats:
                        measurement_type_stats[measurement_type]["count"] += 1
                        measurement_type_stats[measurement_type]["topics"].add(dt_schema["topic"])
                        measurement_type_stats[measurement_type]["dataKeys"].add(data_key)
                    else:
                        measurement_type_stats[measurement_type] = { "count":  1, "topics": {dt_schema["topic"]}, "dataKeys": {data_key}}
                    
df = pd.DataFrame(data_key_stats)
df.head()

In [ ]:
measurement_type_stats

In [ ]:
import json

def serialize_sets(obj):
    if isinstance(obj, set):
        return list(obj)

    return obj


with open('measurement_type_stats.json', 'w') as fp:
    json.dump(measurement_type_stats, fp, default=serialize_sets)

In [ ]:
from glob import glob
import pandas as pd

data_key_stats = []
measurement_type_stats = {}

for dt in types:
    for schema in glob(dt + "/*.schema.json", recursive = True):
        manufacturer = schema.split(os.sep)[2]
        dt = schema.split(os.sep)[3]
        topic = schema.split(os.sep)[4].split(".schema.json")[0]
        with open(schema) as f:
            dt_schema = json.load(f)
            n_data_keys = len(dt_schema["properties"].keys())
            data_key_stats.append({"n_data_keys": n_data_keys, "manufacturer": manufacturer, "device_type": dt, "topic": topic, "device_type_id": manufacturer + "." + dt})

            for data_key, value in dt_schema["properties"].items():
                if value.get("hideFromKpis") is not True:
                    measurement_type = value.get("$ref")
                    if measurement_type is not None:
                        if measurement_type in measurement_type_stats:
                            measurement_type_stats[measurement_type]["count"] += 1
                            measurement_type_stats[measurement_type]["topics"].add(dt_schema["topic"])
                            measurement_type_stats[measurement_type]["dataKeys"].add(data_key)
                        else:
                            measurement_type_stats[measurement_type] = { "count":  1, "topics": {dt_schema["topic"]}, "dataKeys": {data_key}}
                    
df = pd.DataFrame(data_key_stats)
df.head()

In [ ]:
measurement_type_stats

In [ ]:
import json

def serialize_sets(obj):
    if isinstance(obj, set):
        return list(obj)

    return obj


with open('measurement_type_stats_no_hide_from_kpis.json', 'w') as fp:
    json.dump(measurement_type_stats, fp, default=serialize_sets)